## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [4]:
import pandas as pd
df = pd.read_csv('/datasets/data.csv')
print (df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


Всего строк 21524, у двух переменных есть пропущенные значения, это "days_employed" и  "total_income".

### Шаг 2. Предобработка данных

### Обработка пропусков

In [5]:
df.head() # просмотр оглавления

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


Обозначим тип переменных:
children — количество детей в семье - (количественная)
days_employed — общий трудовой стаж в днях - (количественная)
dob_years — возраст клиента в годах - (количественная)
education — уровень образования клиента - (категориальная)
education_id — идентификатор уровня образования - (категориальная)
family_status — семейное положение - (категориальная)
family_status_id — идентификатор семейного положения - (категориальная)
gender — пол клиента - (категориальная)
income_type — тип занятости - (категориальная)
debt — имел ли задолженность по возврату кредитов -(булево)
total_income — ежемесячный доход - (количественная)
purpose — цель получения кредита - (категориальная)

In [6]:
pd.isnull(df).sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [7]:
df['days_employed'].mean()

63046.49766147338

In [8]:
df['days_employed'].median()

-1203.369528770489

In [9]:
df['days_employed'].max()

401755.40047533

Значения "days_employed" некорректны, необходимо обратиться к источнику данных за пояснениями. Для ответа на поставленные вопросы эти данные не требуются, но необходимо сообщить что в них есть ошибка. Возможно, возникновение ошибки как то связано с 'total_income' т.к они имеют одинаковое ко-во пропущенных значений.

In [10]:
print(df['total_income'].mean() ) # вывод значений среднего и медианы
df['total_income'].median()

167422.30220817294


145017.93753253992

In [11]:
total_income_median = df['total_income'].median() # в случае с доходами оптимальной заменой пропущенных значений будет медиана
total_income_median

145017.93753253992

In [12]:
df['total_income'].fillna(total_income_median, inplace = True)

In [13]:
df['total_income'].mean() # проверка на изменение среднего значения

165159.48739726353

In [14]:
pd.isnull(df).sum() # проверка на отсутствие пропусков

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income           0
purpose                0
dtype: int64

В исходных данных обнаружены пропуски, которые были удалены, либо заменены на значения которые характеризуют выборку. Так же по стажу работы обнаружены некорректные значения, информацию необходимо передать заказчику 

### Замена типа данных

In [15]:
df['total_income'] = df['total_income'].astype('int64') # изменение типа данных на целочисленное методом .astype()
df['total_income'].head() # проверка изменения типа данных

0    253875
1    112080
2    145885
3    267628
4    158616
Name: total_income, dtype: int64

Потребовалось измениние типа данных для одного из показателей. Это необходимо т.к. нет необходимости использовать вещественные числа, а с целыми числами работать комфортнее и они задействует меньше памяти


### Обработка дубликатов

In [16]:
df.duplicated().value_counts() # проверка на наличие дубликатов

False    21471
True        54
dtype: int64

In [17]:
df['education'] = df['education'].str.lower() # приведение к нижнему регистру 
df['family_status'] = df['family_status'].str.lower()
df['gender'] = df['gender'].str.lower()
df['income_type'] = df['income_type'].str.lower()
df['purpose'] = df['purpose'].str.lower()
df.duplicated().value_counts() # проверка кол-ва дубликатов после изменения регистра

False    21454
True        71
dtype: int64

In [18]:
df.loc[df.duplicated()] # просмотр списка дубликатов

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
2849,0,NaN,41,среднее,1,женат / замужем,0,f,сотрудник,0,145017,покупка жилья для семьи
3290,0,NaN,58,среднее,1,гражданский брак,1,f,пенсионер,0,145017,сыграть свадьбу
4182,1,NaN,34,высшее,0,гражданский брак,1,f,сотрудник,0,145017,свадьба
4851,0,NaN,60,среднее,1,гражданский брак,1,f,пенсионер,0,145017,свадьба
5557,0,NaN,58,среднее,1,гражданский брак,1,f,пенсионер,0,145017,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
20702,0,NaN,64,среднее,1,женат / замужем,0,f,пенсионер,0,145017,дополнительное образование
21032,0,NaN,60,среднее,1,женат / замужем,0,f,пенсионер,0,145017,заняться образованием
21132,0,NaN,47,среднее,1,женат / замужем,0,f,сотрудник,0,145017,ремонт жилью
21281,1,NaN,30,высшее,0,женат / замужем,0,f,сотрудник,0,145017,покупка коммерческой недвижимости


In [19]:
df_dublicated = df.loc[df.duplicated()]
df_dublicated.groupby('total_income').count()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,purpose
total_income,,,,,,,,,,,
145017,71,0,71,71,71,71,71,71,71,71,71


Просматривая фрейм с дубликатами выяснилось, что во всех случаях одинаковое значение ежемесячного дохода, а так же, отсутствуют данные по стажу. Так же, в столбцах 'days_employed' и 'total_income' одинаковое кол-во пропущеных значений в исходных данных. Возможно, произошла ошибка при выгрузке данных о работе и доходах.

In [20]:
df = df.drop_duplicates() # удалиние дубликатов методом drop_duplicates() как наиболее подходящим и менее трудозатратным
df.loc[df.duplicated()] # проверка на удаление

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose


In [21]:
df['children'].value_counts() # проверка на наличие артефактов

 0     14091
 1      4808
 2      2052
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Присутствуют некорректные значения "20" и "-1". Вероятно, существуют семьи с кол-вом детей 20, но если учитывать данное распредение где кол-во детей равное 4 и 5 встречается в таблице меньшее раз чем кол-во детей равное 20, то можно сделать вывод что значение "20" некорректно во всех 76 случаях. Необходимо уточнить у заказчика каким способом были получены эти данные, если это ручной ввод, то вероятно 20 это 2, а -1 это 1, если это автоматическая выгрузка из проверенных источников, то некорректные значения следует заменить на среднее значение. Используем первый вариант где 20 = 2, а -1 = 1.

In [22]:
df.loc[(df['children'] == 20), 'children'] = 2
df.loc[(df['children'] ==-1), 'children'] = 1
df['children'].value_counts() # проверка изменения данных

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [23]:
df['dob_years'].value_counts() # проверка на наличие артефактов

35    616
40    607
41    605
34    601
38    597
42    596
33    581
39    572
31    559
36    554
44    545
29    544
30    537
48    536
37    536
50    513
43    512
32    509
49    508
28    503
45    496
27    493
52    484
56    483
47    477
54    476
46    472
53    459
57    456
58    454
51    446
55    443
59    443
26    408
60    374
25    357
61    354
62    348
63    269
24    264
64    260
23    252
65    193
22    183
66    182
67    167
21    111
0     101
68     99
69     85
70     65
71     56
20     51
72     33
19     14
73      8
74      6
75      1
Name: dob_years, dtype: int64

Обнаружен 101 запись с возрастом равным нулю. Заменим эти данные наиболее характерным значением для выборки - медианой

In [24]:
df.loc[df['dob_years']== 0, 'dob_years'] = df['dob_years'].median()
df['dob_years'].value_counts()

42.0    697
35.0    616
40.0    607
41.0    605
34.0    601
38.0    597
33.0    581
39.0    572
31.0    559
36.0    554
44.0    545
29.0    544
30.0    537
48.0    536
37.0    536
50.0    513
43.0    512
32.0    509
49.0    508
28.0    503
45.0    496
27.0    493
52.0    484
56.0    483
47.0    477
54.0    476
46.0    472
53.0    459
57.0    456
58.0    454
51.0    446
55.0    443
59.0    443
26.0    408
60.0    374
25.0    357
61.0    354
62.0    348
63.0    269
24.0    264
64.0    260
23.0    252
65.0    193
22.0    183
66.0    182
67.0    167
21.0    111
68.0     99
69.0     85
70.0     65
71.0     56
20.0     51
72.0     33
19.0     14
73.0      8
74.0      6
75.0      1
Name: dob_years, dtype: int64

In [25]:
print (df['education'].value_counts())
df['education_id'].value_counts()

среднее                15172
высшее                  5250
неоконченное высшее      744
начальное                282
ученая степень             6
Name: education, dtype: int64


1    15172
0     5250
2      744
3      282
4        6
Name: education_id, dtype: int64

In [26]:
print (df['family_status'].value_counts())
df['family_status_id'].value_counts()

женат / замужем          12339
гражданский брак          4151
не женат / не замужем     2810
в разводе                 1195
вдовец / вдова             959
Name: family_status, dtype: int64


0    12339
1     4151
4     2810
3     1195
2      959
Name: family_status_id, dtype: int64

In [27]:
df['gender'].value_counts()

f      14174
m       7279
xna        1
Name: gender, dtype: int64

обнаружен артефакт 'xna' в количестве 1  в столбце 'gender'. Необходимо уточнить у заказчика что означает 'xna'. 

In [28]:
df.loc[df['gender']=='xna', 'gender'] = 'f' # замена артефакта на 'f'
df['gender'].value_counts() # проверка

f    14175
m     7279
Name: gender, dtype: int64

In [29]:
df['income_type'].value_counts()

сотрудник          11084
компаньон           5078
пенсионер           3829
госслужащий         1457
предприниматель        2
безработный            2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [30]:
df['debt'].value_counts()

0    19713
1     1741
Name: debt, dtype: int64

In [31]:
df['total_income'].value_counts()

145017    2104
126262       3
160905       3
150684       3
154199       3
          ... 
109583       1
101387       1
138249       1
280240       1
264193       1
Name: total_income, Length: 18606, dtype: int64

2104 одинаковых значений дохода равного 145017. это пропущенные значения заменные медианой

In [32]:
df['purpose'].value_counts()

свадьба                                   791
на проведение свадьбы                     768
сыграть свадьбу                           765
операции с недвижимостью                  675
покупка коммерческой недвижимости         661
операции с жильем                         652
покупка жилья для сдачи                   651
операции с коммерческой недвижимостью     650
жилье                                     646
покупка жилья                             646
покупка жилья для семьи                   638
строительство собственной недвижимости    635
недвижимость                              633
операции со своей недвижимостью           627
строительство жилой недвижимости          624
покупка недвижимости                      621
покупка своего жилья                      620
строительство недвижимости                619
ремонт жилью                              607
покупка жилой недвижимости                606
на покупку своего автомобиля              505
заняться высшим образованием      

артефактов нет, но есть дубли по смысловому значению, а так же необходимо разбить ценли кредита на категории.

Удалены дубликаты с учетом регистра. Обнаружены и обработаны артефакты в кол-ве детей и гендере.

### Лемматизация

In [33]:
import pandas as pd # импортирование библиотеки
feedback=pd.read_csv('/datasets/feedback.csv')
from pymystem3 import Mystem
m = Mystem()


In [34]:
def lemma(purpose): # лемматизация целей получения кредита
    lemmas = m.lemmatize(purpose)
    return lemmas 
   
df['lemmas_purpose']=df['purpose'].apply(lemma)

In [35]:
df['lemmas_purpose'] # вывод лемматизации

0                             [покупка,  , жилье, \n]
1                   [приобретение,  , автомобиль, \n]
2                             [покупка,  , жилье, \n]
3                [дополнительный,  , образование, \n]
4                           [сыграть,  , свадьба, \n]
                             ...                     
21520                  [операция,  , с,  , жилье, \n]
21521               [сделка,  , с,  , автомобиль, \n]
21522                              [недвижимость, \n]
21523    [на,  , покупка,  , свой,  , автомобиль, \n]
21524             [на,  , покупка,  , автомобиль, \n]
Name: lemmas_purpose, Length: 21454, dtype: object

Требуется категоризация данных

### Категоризация данных

In [ ]:
# категоризация по цели получения кредита путем лемитизации

def purpose_categorizer(row): 
    purpose = " ".join(m.lemmatize(row['purpose']))
    
    if 'автом' in purpose:
        return 'автомомбиль'
    if 'образов' in purpose:
        return 'образование'
    elif 'недвиж' in purpose or 'жиль' in purpose:
        return 'недвижимость'
    else:
        return 'свадьба'

df['purpose_category'] = df.apply(purpose_categorizer, axis=1)        
df.head(10)

Так же, требуется категоризация по уровню дохода и по возрасту

In [37]:
pd.qcut(df['total_income'], 5) # выявление 5ти категорий путем разбивки всех значений на 5 групп с одинаковым кол-вом элементом в каждой


0        (214606.0, 2265604.0]
1          (98537.6, 135448.4]
2         (135448.4, 156425.8]
3        (214606.0, 2265604.0]
4         (156425.8, 214606.0]
                 ...          
21520    (214606.0, 2265604.0]
21521     (135448.4, 156425.8]
21522     (20666.999, 98537.6]
21523    (214606.0, 2265604.0]
21524     (20666.999, 98537.6]
Name: total_income, Length: 21454, dtype: category
Categories (5, interval[float64]): [(20666.999, 98537.6] < (98537.6, 135448.4] < (135448.4, 156425.8] < (156425.8, 214606.0] < (214606.0, 2265604.0]]

In [38]:
# категоризация по уровню дохода
def total_income_category(total_income):
      
    if total_income < 98538:
        return 'низкий'
    if total_income < 135448:
        return 'ниже среднего'
    if total_income < 156426:
        return 'средний'
    if total_income < 214606:
        return 'выше среднего'
    else:
        return 'высокий'

df['total_income_category'] = df['total_income'].apply(total_income_category)       
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,purpose_category,total_income_category
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомомбиль,ниже среднего
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,выше среднего
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,высокий
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]",образование,средний
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,низкий
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний


In [39]:
# категоризация по наличию детей

def children_category(children): 
      
    if children > 0:
        return 'дети есть'
    
    else:
        return 'детей нет'

df['children_category'] = df['children'].apply(children_category)       
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,purpose_category,total_income_category,children_category
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий,дети есть
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомомбиль,ниже среднего,дети есть
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний,детей нет
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий,дети есть
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,выше среднего,детей нет
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий,детей нет
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,высокий,детей нет
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]",образование,средний,детей нет
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,низкий,дети есть
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний,детей нет


In [40]:
# категоризация по возрасту (возможно понадобится)
def dob_years_category(dob_years):
      
    if dob_years < 18:
        return 'ВНИМАНИЕ возраст менее 18лет'
    if dob_years <=35 :
        return 'молодеж'
    if dob_years <=65:
        return 'средний возраст'
    else:
        return 'пенсионер'

df['dob_years_category'] = df['dob_years'].apply(dob_years_category)       
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas_purpose,purpose_category,total_income_category,children_category,dob_years_category
0,1,-8437.673028,42.0,высшее,0,женат / замужем,0,f,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий,дети есть,средний возраст
1,1,-4024.803754,36.0,среднее,1,женат / замужем,0,f,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомомбиль,ниже среднего,дети есть,средний возраст
2,0,-5623.422610,33.0,среднее,1,женат / замужем,0,m,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",недвижимость,средний,детей нет,молодеж
3,3,-4124.747207,32.0,среднее,1,женат / замужем,0,m,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,высокий,дети есть,молодеж
4,0,340266.072047,53.0,среднее,1,гражданский брак,1,f,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,выше среднего,детей нет,средний возраст
5,0,-926.185831,27.0,высшее,0,гражданский брак,1,m,компаньон,0,255763,покупка жилья,"[покупка, , жилье, \n]",недвижимость,высокий,детей нет,молодеж
6,0,-2879.202052,43.0,высшее,0,женат / замужем,0,f,компаньон,0,240525,операции с жильем,"[операция, , с, , жилье, \n]",недвижимость,высокий,детей нет,средний возраст
7,0,-152.779569,50.0,среднее,1,женат / замужем,0,m,сотрудник,0,135823,образование,"[образование, \n]",образование,средний,детей нет,средний возраст
8,2,-6929.865299,35.0,высшее,0,гражданский брак,1,f,сотрудник,0,95856,на проведение свадьбы,"[на, , проведение, , свадьба, \n]",свадьба,низкий,дети есть,молодеж
9,0,-2188.756445,41.0,среднее,1,женат / замужем,0,m,сотрудник,0,144425,покупка жилья для семьи,"[покупка, , жилье, , для, , семья, \n]",недвижимость,средний,детей нет,средний возраст


In [41]:
df['dob_years_category'].value_counts()

средний возраст    14169
молодеж             6583
пенсионер            702
Name: dob_years_category, dtype: int64

Категоризация приводит данные в читаемый вид, готовый для исследования и ответа на поставленные вопросы.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [42]:
children_pivot = df.pivot_table('debt', index= 'children_category', margins = True)
children_pivot

,debt
children_category,
детей нет,0.075438
дети есть,0.092082
All,0.081150


In [43]:
df['debt'].sum()/len(df['debt']) #проверка полученного значения All

0.08115036822970076

In [44]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

Доля должников с просрочкой по кредиту у которых есть дети равена 9.2%
Доля должников с просрочкой по кредиту у которых нет детей равна 7.5%
Наблюдается разница в 1.7% между двумя категориями должников, можно утвержать что присутствует незначительная зависимость между наличием детей и возвратом кредита в срок.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [45]:
family_status_pivot = df.pivot_table('debt', index= 'family_status', margins = True)
family_status_pivot



,debt
family_status,
в разводе,0.071130
вдовец / вдова,0.065693
гражданский брак,0.093471
женат / замужем,0.075452
не женат / не замужем,0.097509
All,0.081150


Семейное положение влияет на возврат кредита в срок. Не замужние/женатые, а так же клиенты находящиеся в гражданском браке чаще других допускают просрочки по платежам. Самымы недежными являются клиенты в разводе и вдовец/вдова

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [46]:
total_income_pivot = df.pivot_table('debt', index= 'total_income_category', margins = True)
total_income_pivot

,debt
total_income_category,
высокий,0.069914
выше среднего,0.086227
ниже среднего,0.082751
низкий,0.080168
средний,0.086693
All,0.081150


Заемщик с высоким уровнем дохода является ниболее надежным. В то же время заемщик с низким уровнем дохода не является более рискованным клиентом относительно "средних" доходов т.к. доля заемщиков с низким доходом и просрочкой даже ниже чем у тех, у кого доход выше.  

- Как разные цели кредита влияют на его возврат в срок?

In [47]:
purpose_pivot = df.pivot_table('debt', index= 'purpose_category', margins = True)
purpose_pivot

,debt
purpose_category,
автомомбиль,0.093590
недвижимость,0.072334
образование,0.092200
свадьба,0.080034
All,0.081150


Цель получения кредита влияет на вероятность возникновения просроченной задолженности. Самой надежной целью в плане возврата кредита в срок является приобритение недвижимости, наименее надежными - приобритение автомобиля и образование.

In [48]:
most_unreliable = df.loc[(df['purpose_category']=='автомомбиль') & (df['total_income_category']=='средний') & (df['family_status']== 'не женат / не замужем') & (df['children_category']=='дети есть') ]

In [49]:
print((most_unreliable['debt'].sum() / len(most_unreliable['debt']) * 100),"% клиентов которые имеют одновременно все негативно влияющие факторы не возвращяют кредит в срок")
print ("Для сравнения, общая доля кредитов не погашенных в срок равна", "%.1f" % (df['debt'].sum() / len(df['debt']) * 100), "%")

20.0 % клиентов которые имеют одновременно все негативно влияющие факторы не возвращяют кредит в срок
Для сравнения, общая доля кредитов не погашенных в срок равна 8.1 %


### Шаг 4. Общий вывод

Исходные данные имеют ряд ошибок в виде пропусков, дубликатов и артефактов. После обработки данные пригодны для исследования, но необходимо обозначить заказчику наличие ошибок и возможные причины их возникновения. 
Найдена взаимосвязь между возвратом кредита в срок и такими данными как наличие детей, уровень дохода, цель кредита и семейное положение клиента. 
Пересекающиеся негативные факторы увеличивают риск получить ненадежного клиента.
Возможен дальнейший анализ взаимосвязей связей между факторами влияющими на возврат кредита в срок.